# EarthNet2021 Model Intercomparson Suite Cookbook

Run.py is the main entry point. You should be able to launch any task from it! But first let's...

## Download EarthNet2021 Dataset

In [ ]:
import earthnet as en

en.Downloader.get('data/datasets/release', 'all') #["train","iid","ood","extreme","seasonal"]

## Test Baseline

In [ ]:
!python run.py --model_name Baseline --split_name iid_test_split --submodel mean

## Train
The EN Intercomparison suite comes with a Tensorflow baseline based on SAVP that we call Arcon-STF (Autorgressive Conditional Spatio-Temporal Forecaster).
You can train and test any model using run.py. Checkout 'configs/tf_template/' to learn how the standalone model is passed arguments from run.py.

In [ ]:
# Firt we generate tfrecords samples for the train set using the included util
!source activate ENtf115py36; python utils/npz_to_tfecords.py --inpath data/datasets/release/train/ --outpath data/tf_dataset/train

In [ ]:
# Running train Arcon trough run.py

!python run.py --model_name tf_template --dataroot data/tf_dataset/ --split_name train --experiment_name sample_hparams_Arcon-STF --mode train --submodel arcon_earthnet --dataset earthnet --gpu_ids 0 --experiment_settings sample_hparams_Arcon-STF

In [ ]:
#To continue training a paused train run

!python run.py --model_name tf_template --dataroot data/tf_dataset/ --split_name train --experiment_name sample_hparams_Arcon-STF --mode train --submodel arcon_earthnet --dataset earthnet --gpu_ids 0 --resume

## Test

In [ ]:
# We generate tfrecords samples for the iid_test set using the included util
!source activate ENtf115py36; python utils/npz_to_tfecords.py --inpath data/datasets/release/iid_test_split/ --outpath data/tf_dataset/iid_test_split

In [ ]:
#Predicting tf_template's Arcon model

!python run.py --model_name tf_template --dataroot data/tf_dataset/ --split_name iid --experiment_name sample_hparams_Arcon-STF --mode test --submodel arcon_earthnet --dataset earthnet --gpu_ids 0 --num_stochastic_samples 1

In [ ]:
#Predicting Baseline model linear regression

!python run.py --model_name Baseline --split_name iid_test_split --submodel mean

## Evaluate

In [ ]:
#Evaluates the output of the model Baseline
!python run.py --mode evaluate --model_name Baseline --experiment_name mean

In [ ]:
#Evaluates the output of the tf_template Arcon
!python run.py --mode evaluate --model_name tf_template --split_name iid_test_split --experiment_name sample_hparams_Arcon-STF 

In [ ]:
#plotting the figures to display evaluation results
# use --animations to create videos of [0,0.15,0.3,0.5,0.7,0.85,1] quantile samples by model performance.
!python run.py --mode plot
#Find them in data/results/figures

## Plot
Quick check on how to use the utility to plot multicubes in utils/plot_sample.py

In [ ]:
# Plotting a single frame from NPZ with predictions

!python utils/plot_sample.py --pred --frame 150 --image

In [ ]:
# Plotting a .mp4 video

!python utils/plot_sample.py --pred --frame 150 --video --tilename 29SND --filename 29SND_2017-06-10_2017-11-06_2105_2233_2873_3001_32_112_44_124.npz --datapath data/datasets/release/train

#Plotting samples on the test sets (whose groundtruth is split into context and target files is not yet working)